# 인코더 LSTM과 디코더 LSTM(Sequence to Sequence)로 번역기 만들기
- Google Neural MAchine Translation(GNMT)
- RNN 기반의 Seq2Seq방식
- 자연어 생성 알고리즘

## 1. 패키지 import 하이퍼파라미터
- 하이퍼파라미터 : 모델의 정확도 및 학습속도에 영향을 미치는 변수

In [2]:
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Input, Dense, LSTM
from keras.utils import to_categorical

In [102]:
MY_HIDDEN = 128
MY_EPOCH = 500

## 2 번역 데이터 가져오기

In [ ]:
raw = pd.read_csv('data/translate.csv', header=None)
eng_kor = raw.values.tolist()
print('영어-한글 번역 데이터 :', eng_kor[:3])
print('영어-한글 번역 데이터 수', len(eng_kor))

영어-한글 번역 데이터 : [['cold', '감기'], ['come', '오다'], ['cook', '요리']]


## 3. 영어 알파벳과 한글문자 리스트 만들기

In [32]:
# 영어 알파벳 리스트
e_alpha = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz']

# 한글문자 리스트
korean = ''.join([data[1] for data in eng_kor])
k_ch = list(set([ch for ch in korean]))
k_ch.sort()
k_ch

k_alpha = pd.read_csv('data/korean.csv', header=None)[0].tolist()
print(k_alpha)

['가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']


In [38]:
alpha = e_alpha + k_alpha
print('영어와 한글 알파벳 :',alpha)
alpha_total_size = len(alpha)
print('전체 알파벳 갯수(원핫인코딩할 size) :', alpha_total_size)

영어와 한글 알파벳 : ['S', 'E', 'P', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '가', '각', '간', '감', '개', '거', '것', '게', '계', '고', '관', '광', '구', '굴', '규', '그', '금', '기', '깊', '나', '날', '남', '내', '넓', '녀', '노', '놀', '농', '높', '뉴', '늦', '다', '단', '도', '동', '들', '람', '랑', '래', '램', '류', '름', '릎', '리', '많', '망', '매', '머', '먼', '멍', '메', '명', '모', '목', '무', '물', '미', '바', '반', '방', '번', '복', '부', '분', '붕', '비', '뿌', '사', '상', '색', '생', '서', '선', '소', '손', '수', '쉽', '스', '시', '식', '실', '싸', '아', '약', '얇', '어', '언', '얼', '여', '연', '오', '옥', '왼', '요', '용', '우', '운', '움', '위', '유', '은', '을', '음', '의', '이', '익', '인', '읽', '입', '자', '작', '장', '적', '제', '좋', '주', '지', '짜', '쪽', '찾', '책', '출', '칙', '크', '키', '탈', '택', '통', '파', '팔', '편', '피', '핑', '한', '합', '해', '행', '험', '회', '획', '휴', '흐']
전체 알파벳 갯수(원핫인코딩할 size) : 171


## 4. 문자당 num를 갖는 dict 만들기

In [124]:
char_to_num = {c:i for i, c in enumerate(alpha)}
num_to_char = {i:c for i, c in enumerate(alpha)}
print(char_to_num)

{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '가': 29, '각': 30, '간': 31, '감': 32, '개': 33, '거': 34, '것': 35, '게': 36, '계': 37, '고': 38, '관': 39, '광': 40, '구': 41, '굴': 42, '규': 43, '그': 44, '금': 45, '기': 46, '깊': 47, '나': 48, '날': 49, '남': 50, '내': 51, '넓': 52, '녀': 53, '노': 54, '놀': 55, '농': 56, '높': 57, '뉴': 58, '늦': 59, '다': 60, '단': 61, '도': 62, '동': 63, '들': 64, '람': 65, '랑': 66, '래': 67, '램': 68, '류': 69, '름': 70, '릎': 71, '리': 72, '많': 73, '망': 74, '매': 75, '머': 76, '먼': 77, '멍': 78, '메': 79, '명': 80, '모': 81, '목': 82, '무': 83, '물': 84, '미': 85, '바': 86, '반': 87, '방': 88, '번': 89, '복': 90, '부': 91, '분': 92, '붕': 93, '비': 94, '뿌': 95, '사': 96, '상': 97, '색': 98, '생': 99, '서': 100, '선': 101, '소': 102, '손': 103, '수': 104, '쉽': 105, '스': 106, '시': 107, '식': 108, '실': 109, '싸': 110,

In [51]:
data = eng_kor[0]
print('인코더 입력(원핫인코딩전) :',[char_to_num[ch] for ch in data[0]])
print('디코더 입력(원핫인코딩전) :',[char_to_num[ch] for ch in 'S'+data[1]])
print('디코더 출력(원핫인코딩X) :', [char_to_num[ch] for ch in data[1]+'E'])

인코더 입력(원핫인코딩전) : [5, 17, 14, 6]
디코더 입력(원핫인코딩전) : [0, 32, 46]
디코더 출력(원핫인코딩X) : [32, 46, 1]


In [52]:
# 희소행렬의 원핫인코딩 방법1 (희소행렬에소는 pd.get_dummies([2,9,7]) 안씀)
to_categorical([char_to_num[ch] for ch in data[0]], num_classes=alpha_total_size)

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.,

In [54]:
# 희소행렬의 원핫인코딩 방법2 
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

## 5. 인코더 입력, 디코더 입력, 디코더 출력
- 인코더 입력 : 영어 알파벳 -> 숫자 -> 원핫인코딩(110,4,171)
- 디코더 입력 : 'S'+한글문자 -> 숫자 -> 원핫인코딩(110,4,171)
- 디코더 출력 : 한글문자+'E' -> 숫자(110, 3)인 list -> 110,3,1 배열로

In [98]:
def encoding(eng_kor = eng_kor):
    enc_in = []
    dec_in = []
    dec_out = []
    for data in eng_kor:
        # 인코더 입력데이터(영어알파벳 -> 숫자 _> 원핫인코딩)
        eng = [char_to_num[c] for c in data[0]]
        eng_one = np.eye(alpha_total_size)[eng]
        enc_in.append(eng_one)

        # 디코더 입력데이터('S'+한글 -> 숫자 -> 원핫인코딩)
        kor = [char_to_num[c] for c in 'S'+data[1]]
        kor_one = np.eye(alpha_total_size)[kor]
        dec_in.append(kor_one)
        
        # 디코더 출력데이터(한글+'E' -> 숫자)
        kor = [char_to_num[c] for c in data[1]+'E']
        dec_out.append(kor)

    return enc_in, dec_in, dec_out


ei, di, do = encoding([['cold', '감기'],['come', '오다'],['cook', '요리'],['copy', '복사']])
X_enc = np.array(ei)
X_dec = np.array(di)
Y_dec = np.array(do)

In [ ]:
# 축증가 방법1
Y_dec = np.array(do).reshape(-1,3,1)
Y_dec.shape

(4, 3, 1)

In [ ]:
# 축증가 방법2 : 맨마지막 축 증가하는 함수
np.expand_dims(Y_dec, axis=-1)

array([[[ 32],
        [ 46],
        [  1]],

       [[119],
        [ 60],
        [  1]],

       [[122],
        [ 72],
        [  1]],

       [[ 90],
        [ 96],
        [  1]]])

In [97]:
# 축증가 방법3
Y_dec[...,np.newaxis]

array([[[ 32],
        [ 46],
        [  1]],

       [[119],
        [ 60],
        [  1]],

       [[122],
        [ 72],
        [  1]],

       [[ 90],
        [ 96],
        [  1]]])

In [99]:
# 축증가 방법4
Y_dec[:,:,None]

array([[[ 32],
        [ 46],
        [  1]],

       [[119],
        [ 60],
        [  1]],

       [[122],
        [ 72],
        [  1]],

       [[ 90],
        [ 96],
        [  1]]])

## 6. 전체 입력데이터, 타겟데이터 준비

In [100]:
x_enc, x_dec, y_dec = encoding(eng_kor)
X_enc = np.array(x_enc)
X_dec = np.array(x_dec)
Y_dec = np.array(y_dec).reshape(-1,3,1)

X_enc.shape, X_dec.shape, Y_dec.shape

((110, 4, 171), (110, 3, 171), (110, 3, 1))

## 7 모델 구현

In [113]:
# 인코더 LSTM

ENC_IN = Input(shape=(4,alpha_total_size))

_, state_h, state_c = LSTM(units=MY_HIDDEN, return_state=True)(ENC_IN)

# 인코더와 디코더 연결 고리
LINK = [state_h, state_c]

# 디코더 LSTM
DEC_IN = Input(shape=(3,alpha_total_size))
DEC_MID = LSTM(units=MY_HIDDEN, 
               return_sequences=True # 윗 출력 받음
               )(DEC_IN, initial_state=LINK)

DEC_OUT = Dense(units=alpha_total_size, activation='softmax')(DEC_MID)

# 모델
model = Model(inputs = [ENC_IN,DEC_IN],
              outputs = DEC_OUT)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 4, 171)]     0           []                               
                                                                                                  
 input_15 (InputLayer)          [(None, 3, 171)]     0           []                               
                                                                                                  
 lstm_11 (LSTM)                 [(None, 128),        153600      ['input_14[0][0]']               
                                 (None, 128),                                                     
                                 (None, 128)]                                                     
                                                                                              

## 8. 학습과정 설정 & 학습

In [114]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [115]:
hist = model.fit([X_enc, X_dec], Y_dec, epochs=MY_EPOCH)

Epoch 1/500
4/4 [==============================] - 4s 11ms/step - loss: 5.1210 - accuracy: 0.2000
Epoch 2/500
4/4 [==============================] - 0s 12ms/step - loss: 4.9823 - accuracy: 0.3333
Epoch 3/500
4/4 [==============================] - 0s 11ms/step - loss: 4.2604 - accuracy: 0.3333
Epoch 4/500
4/4 [==============================] - 0s 10ms/step - loss: 3.4540 - accuracy: 0.3333
Epoch 5/500
4/4 [==============================] - 0s 11ms/step - loss: 3.4036 - accuracy: 0.3333
Epoch 6/500
4/4 [==============================] - 0s 11ms/step - loss: 3.3677 - accuracy: 0.3333
Epoch 7/500
4/4 [==============================] - 0s 11ms/step - loss: 3.3381 - accuracy: 0.3333
Epoch 8/500
4/4 [==============================] - 0s 9ms/step - loss: 3.3160 - accuracy: 0.3333
Epoch 9/500
4/4 [==============================] - 0s 10ms/step - loss: 3.2881 - accuracy: 0.3333
Epoch 10/500
4/4 [==============================] - 0s 10ms/step - loss: 3.2841 - accuracy: 0.3333
Epoch 11/500
4/4 [==

## 9. 모델사용

In [119]:
# 쉬운문제
easy_test = [['loss','PP'],
             ['male','PP'],
             ['luck','PP'],
             ['milk','PP']]
enc_in, dec_in, _ =encoding(easy_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
enc_in.shape, dec_in.shape

((4, 4, 171), (4, 3, 171))

In [139]:
# 위의 문제 예측하기
pred = model.predict([enc_in,dec_in])
pred = pred.argmax(axis=-1)
for test,yhat in zip(easy_test,pred):
    word = ''.join([num_to_char[char] for char in yhat[:-1]])
    print( test[0], ':',word)
   

1/1 [==============================] - 0s 41ms/step
loss : 손해
male : 남자
luck : 행운
milk : 우유


In [144]:
# 어려운 문제
hard_test = [['lsos','PP'],
             ['mlae','PP'],
             ['luck','PP'],
             ['mlik','PP']]
enc_in, dec_in, _ =encoding(hard_test)
enc_in = np.array(enc_in)
dec_in = np.array(dec_in)
pred = model.predict([enc_in,dec_in]).argmax(axis=-1)
for test,yhat in zip(hard_test,pred):
    word = ''.join([num_to_char[char] for char in yhat[:-1]])
    print( test[0], ':',word)

1/1 [==============================] - 0s 27ms/step
lsos : 손금
mlae : 남자
luck : 행운
mlik : 우유
